# Spark assignment 2: Collocations

As for the second part of the assignment, your task is to extract collocations: that is word combinations that occur together. For example, “high school” or “roman empire”.

To find collocations, you will use NPMI (normalized pointwise mutual information) metric.

PMI of two words, a & b, is defined as “PMI(a, b) = ln (P(ab) / (P(a) * P(b))”, where P(ab) is the probability of two words coming one after the other, and P(a) and P(b) are probabilities of words a & b respectively.

You will estimate probabilities with occurrence counts, that is “P(a) = # of occurrences of word a / total number of words”, and “P(ab) = # of occurrences of words ‘a b’ / total number of word pairs”.

To build an intuition behind the definition, see Reading material.

Therefore, rare combinations of coupled words have large PMI.

NPMI is computed as “NPMI(a, b) = PMI(a, b) / -ln P(ab)”. This normalizes the quantity to be within the range [-1; 1].

In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[*]"))

Extract all the words.

In [2]:
import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
    except ValueError as e:
        return []
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    return words

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)

Filter out stopwords using the dictionary (/datasets/stop_words_en.txt ) (do not forget to convert words to the lowercase!)

In [3]:
with open('/datasets/stop_words_en.txt') as f1:
    stop_words = set(f1.read().split())
    
def flt(words):
    res = list()
    for item in words:
        if item not in stop_words:
            res.append(item)
    return res

wiki_lower = wiki.map(lambda words: [x.lower() for x in words])
wiki_filtered = wiki_lower.map(lambda words: flt(words))

Compute all bigrams (that is, pairs of consequent words)

In [4]:
def make_bigrams(words):
    result = list()
    
    for i, word in enumerate(words[:-1]):
        result.append((word + '_' + words[i + 1], 1))
    
    return result

bigrams = wiki_filtered.flatMap(lambda item: make_bigrams(item))

Leave only bigrams with at least 500 occurrences

In [5]:
baigrams_occurrence = bigrams.reduceByKey(lambda accum, n: accum + n)
filtered_bigrams = baigrams_occurrence.filter(lambda (bigram, count): count >= 500)

Total words and bigrams count

In [6]:
total_words = wiki_filtered.flatMap(lambda words: [item for item in words]).count()
total_bigrams = bigrams.count()

Words occurance

In [7]:
words_occurrence_rdd = wiki_filtered.flatMap(lambda words: [(word, 1) for word in words])\
                                    .reduceByKey(lambda accum, n: accum + n)

words_prob = dict()

for (word, count) in words_occurrence_rdd.collect():
    words_prob[word] = float(count) / total_words

Bigrams probability

In [8]:
bigrams_prob = filtered_bigrams.map(lambda (bigram, count): (bigram, float(count) / total_bigrams))

Compute NPMI for every bigram (note: when computing probabilities, you need unpruned counts!)

In [9]:
from math import log

def calc_npmi((bigram, prob)):
    [left_word, right_word] = bigram.split("_", 1)
    left_prob = words_prob[left_word]
    right_prob = words_prob[right_word]
    
    pmi = log(prob / (left_prob * right_prob))
    npmi = pmi / -log(prob)
    
    return (npmi, bigram)

NPMI = bigrams_prob.map(lambda item: calc_npmi(item))

Sort word pairs by NPMI in the descending order

In [10]:
sorted_NPMI = NPMI.sortByKey(False)

Print top 39 word pairs, with words delimited by the underscore “_”

In [11]:
for (npmi, bigram) in sorted_NPMI.take(39):
    print bigram

los_angeles
external_links
united_states
prime_minister
san_francisco
et_al
new_york
supreme_court
19th_century
20th_century
references_external
soviet_union
air_force
baseball_player
university_press
roman_catholic
united_kingdom
references_reading
notes_references
award_best
north_america
new_zealand
civil_war
catholic_church
world_war
war_ii
south_africa
took_place
roman_empire
united_nations
american_singer-songwriter
high_school
american_actor
american_actress
american_baseball
york_city
american_football
years_later
north_american
